# Networks and their Structure Assignment

## Network Science Topic 3

Note that the networks in this exercise are all undirected.

We recall some definitions and introduce some new ones.

Let $d_{ij}$ be the distance (the length of the shortest path) between vertices $i$ to $j$.

Then the *closeness centrality* of vertex $j$ is    $\displaystyle{ \mbox{CC}(j)= \frac{1}{\sum_i d_{ij}}}$.

The *nearness centrality* of vertex $j$ is $\displaystyle{ \mbox{NC}(j)= \sum_i  \frac{1}{d_{ij}}}$.  In both these definitions, the sums are over all vertices $i$, $i \neq j$, in the network.

The *degree centrality* of vertex $j$ is simply its degree (the number of neighbours it has) and is denoted $\mbox{DC}(j)$.

The *adjacency centrality* of vertex $j$ is $ \mbox{AC}(j)=\displaystyle{ \frac{1}{d_j} \sum_i \frac{d_j - d_i}{d_j+d_i} }$ where the sum is over all vertices $i$ that are adjacent to $j$ and $d_i$ denotes the degree of a vertex.  (So $\mbox{DC}(j)$ and $d_j$ are different notations for the same measure.)

1. [5 marks]  Calculate the values of the four centrality measures defined above on each vertex in the network below.  (The diagram and the dictionary are two representations of the same network.)   Present your answer as four lists --- one for each centrality measure --- that gives the vertices and the calculated values ordered by those values.


```python
network = {1:  [4],
           2:  [4],
           3:  [4],
           4:  [1, 2, 3, 5, 6],
           5:  [4],
           6:  [4, 7, 8, 9, 10, 11],
           7:  [6, 8, 11],
           8:  [6, 7, 9, 11],
           9:  [6, 8, 10],
           10: [6, 9, 11, 12],
           11: [6, 7, 8, 10],
           12: [10]}
           
```

<img src="example.jpg" width="400">


In [1]:
from typing import Dict, Set, Optional, Tuple, List, Generator, Hashable, Literal, Union
import itertools
import tqdm

Node = Vertex = Hashable
Network = Graph = Dict[Node, Set[Node]]

In [2]:
def bfs(
        graph_: Graph,
        start_node_: Node
) -> Generator[Tuple[Node, int], None, None]:
    visited_: Set[Node] = {start_node_}
    queue_: List[Tuple[Node, int]] = [(start_node_, 0)]

    while queue_:
        node_, distance_ = queue_.pop(0)

        yield node_, distance_

        for neighbour_ in graph_[node_]:
            if neighbour_ not in visited_:
                visited_.add(neighbour_)
                queue_.append((neighbour_, distance_ + 1))


def distance(graph_: Graph, start_node_: Node, end_node_: Node) -> int:
    for node_, distance_ in bfs(graph_, start_node_):
        if node_ == end_node_:
            return distance_
    return -1


def distances(graph_: Graph, start_node_: Node) -> Dict[int, int]:
    return {node_: distance_ for node_, distance_ in bfs(graph_, start_node_)}


def all_distances(graph_: Graph) -> Dict[Tuple[Node, Node], int]:
    return {(start_node_, end_node_): distance_
            for start_node_ in tqdm.tqdm(sorted(list(graph_)))
            for end_node_, distance_ in sorted(list(distances(graph_, start_node_).items()))}


def closeness_centralities(
        graph_: Graph,
        graph_distances_: Optional[Dict[Tuple[Node, Node], int]] = None
) -> Dict[Node, float]:
    if graph_distances_ is None:
        graph_distances_ = all_distances(graph_)
    return {
        node_: 1.0 / sum([
            graph_distances_[(node_, other_node_)]
            for other_node_ in graph_
            if node_ != other_node_
        ])
        for node_ in graph_
    }


def nearness_centralities(
        graph_: Graph,
        graph_distances_: Optional[Dict[Tuple[Node, Node], int]] = None
) -> Dict[Node, float]:
    if graph_distances_ is None:
        graph_distances_ = all_distances(graph_)
    return {
        node_: sum([
            1.0 / graph_distances_[(node_, other_node_)]
            for other_node_ in graph_
            if node_ != other_node_
        ])
        for node_ in graph_
    }


def degree_centralities(graph_: Graph) -> Dict[Node, int]:
    return {
        node_: len(graph_[node_])
        for node_ in graph_
    }


def adjacency_centralities(
        graph_: Graph,
        degree_centralities_: Optional[Dict[Node, int]] = None
) -> Dict[Node, float]:
    if degree_centralities_ is None:
        degree_centralities_ = degree_centralities(graph_)
    return {
        node_: (1.0 / degree_centralities_[node_]) * sum([
            (degree_centralities_[node_] - degree_centralities_[adjacent_node_]) /
            (degree_centralities_[node_] + degree_centralities_[adjacent_node_])
            for adjacent_node_ in graph_[node_]
        ])
        for node_ in graph_
    }

In [3]:
def compute_centralities(
        network_: Network
) -> Tuple[Dict[Node, float], Dict[Node, float], Dict[Node, int], Dict[Node, float]]:
    # Compute the distance from every node to every other node in the network
    d_ = all_distances(network_)

    # Compute the four centrality measure values for each vertex
    ccs_ = closeness_centralities(network_, d_)
    ncs_ = nearness_centralities(network_, d_)
    dcs_ = degree_centralities(network_)
    acs_ = adjacency_centralities(network_, dcs_)

    return ccs_, ncs_, dcs_, acs_


def format_centrality(
        centralities_: List[Tuple[Node, Union[int, float]]],
        n_: int = 20,
        precision_: int = 4,
        n_max_: int = 30
) -> str:
    n_unique_: int = n_
    while (
            len(set(list(zip(*centralities_[:n_unique_]))[1])) < n_ and
            n_unique_ < len(centralities_) and
            n_unique_ < n_max_
    ): n_unique_ += 1
    return "\n".join([
        f"{i_ + 1:<5}{node_:^20} - {centrality_:.{precision_}f}"
        for i_, (node_, centrality_) in enumerate(centralities_[:n_unique_])
    ])


# noinspection PyTypeChecker
def sort_centrality(centrality_: Dict[Node, Union[int, float]]) -> List[Tuple[Node, Union[int, float]]]:
    return list(map(lambda n_: (n_, centrality_[n_]), sorted(centrality_, key=centrality_.get, reverse=True)))


def format_centralities(
        ccs_: Dict[Node, float],
        ncs_: Dict[Node, float],
        dcs_: Dict[Node, int],
        acs_: Dict[Node, float],
        n_: int = 20,
        n_max_: int = 40
) -> str:
    cc_top_20_ = sort_centrality(ccs_)
    nc_top_20_ = sort_centrality(ncs_)
    dc_top_20_ = sort_centrality(dcs_)
    ac_top_20_ = sort_centrality(acs_)
    return (f"Closeness Centrality:\n"
            f"{format_centrality(cc_top_20_, n_, 6, n_max_)}\n\n"
            f"Nearness Centrality:\n"
            f"{format_centrality(nc_top_20_, n_, 6, n_max_)}\n\n"
            f"Degree Centrality:\n"
            f"{format_centrality(dc_top_20_, n_, 0, n_max_)}\n\n"
            f"Adjacency Centrality:\n"
            f"{format_centrality(ac_top_20_, n_, 6, n_max_)}\n")

In [4]:
network: Network = {
    1: {4},
    2: {4},
    3: {4},
    4: {1, 2, 3, 5, 6},
    5: {4},
    6: {4, 7, 8, 9, 10, 11},
    7: {6, 8, 11},
    8: {6, 7, 9, 11},
    9: {6, 8, 10},
    10: {6, 9, 11, 12},
    11: {6, 7, 8, 10},
    12: {10}
}

ccs, ncs, dcs, acs = compute_centralities(network)

# Print the computed values in a table
print(
    "j\t|\tCC(j)\tNC(j)\tDC(j)\tAC(j)\n" +
    "-" * 4 + "|" + "-" * 35 + "\n" +
    "\n".join([
        f"{j}\t|\t"
        f"{ccs[j]:.4f}\t"
        f"{ncs[j]:.4f}\t"
        f"  {dcs[j]}\t\t"
        f"{acs[j]:.4f}"
        for j in network
    ])
)

100%|██████████| 12/12 [00:00<00:00, 12018.06it/s]

j	|	CC(j)	NC(j)	DC(j)	AC(j)
----|-----------------------------------
1	|	0.0357	4.9167	  1		-0.6667
2	|	0.0357	4.9167	  1		-0.6667
3	|	0.0357	4.9167	  1		-0.6667
4	|	0.0556	7.8333	  5		0.5152
5	|	0.0357	4.9167	  1		-0.6667
6	|	0.0625	8.5000	  6		0.2263
7	|	0.0417	6.1667	  3		-0.2063
8	|	0.0435	6.6667	  4		0.0214
9	|	0.0435	6.3333	  3		-0.2063
10	|	0.0455	6.8333	  4		0.1357
11	|	0.0455	6.8333	  4		-0.0143
12	|	0.0312	4.5000	  1		-0.6000


2. [20 marks] Obtain the three datasets in topic3networks.zip (under Topic 3 on Learn Ultra, see the descriptions below).  Load these networks.  Again, they are all undirected.  We wish also to work with connected graphs so find the largest connected component of each and discard other vertices.

 For each dataset, for each of the four centrality measures, list, in order, the 20 vertices with the highest values of that measure (include more if the values are tied).  Comment on whether you think, based on what you have found, that nearness centrality is a good alternative to closeness centrality and that adjacency centrality is a good alternative to degree centrality.

The datasets:
* london_transport_raw_edges.txt:  The network is of London rail and underground stations that are linked if they are adjacent on some line.  The second and third items on each line in the file are a pair of nodes that are joined by an edge (the first item describes how they are linked and can be ignored for this exercise).
* Roget.txt: This is a network of words that are linked if they appear together in a thesaurus.  At the start of the file is a list of words (the nodes) and their numeric identifiers.    Then there are lists (one per line) of words that appear together in the thesaurus.  There should be an edge between any pair of nodes that appear in the same list.  For example, the list 3 4 323 325 implies the existence of six edges: (3,4), (3, 323), (3, 325), (4, 323), (4, 325), (323, 325)
* CCSB-Y2H.txt: The network is of interactions amongst proteins in yeast (living cells can be considered as complex webs of macromolecular interactions known as interactome networks).  The first two items on each line are a pair of nodes joined by
an edge (the rest of the line can be ignored).

In [5]:
def compute_largest_connected_component(graph_: Graph) -> Set[Node]:
    largest_connected_component_: Set[Node] = set()
    visited_: Set[Node] = set()
    for current_node_ in tqdm.tqdm(list(graph_)):
        if current_node_ not in visited_:
            current_connected_component_ = set(map(lambda n: n[0], bfs(graph_, current_node_)))
            visited_.update(current_connected_component_)
            if len(current_connected_component_) > len(largest_connected_component_):
                largest_connected_component_ = current_connected_component_
    return largest_connected_component_


def remove_nodes(graph_: Graph, nodes_: Set[Node]) -> Graph:
    return {
        node_: neighbours_ - nodes_
        for node_, neighbours_ in graph_.items()
        if node_ not in nodes_
    }


def keep_nodes(graph_: Graph, nodes_: Set[Node]) -> Graph:
    return {
        node_: neighbours_ & nodes_
        for node_, neighbours_ in tqdm.tqdm(graph_.items())
        if node_ in nodes_
    }

In [6]:
def load_london_transport(filename_: str) -> Network:
    network_: Network = {}
    with open(filename_, "rt") as file_:
        for line_ in file_:
            _, left_node_, right_node_ = line_.rstrip().split()
            if left_node_ not in network_:
                network_[left_node_] = set()
            if right_node_ not in network_:
                network_[right_node_] = set()
            if left_node_ != right_node_:
                network_[left_node_].add(right_node_)
                network_[right_node_].add(left_node_)
    return network_


london_transport_network: Network = load_london_transport("./topic3networks/london_transport_raw.edges.txt")

In [7]:
def load_roget(filename_: str) -> Network:
    network_: Network = {}
    with open(filename_, "rt") as file_:
        state_: Literal["vertex-count", "vertex-ids", "arcs-title", "arcs-list"] = "vertex-count"
        vertex_count_: int = 0
        vertex_names_: Dict[int, str] = {}
        for line_ in file_:
            line_: str = line_.rstrip()
            if state_ == "vertex-count":
                left_, right_ = line_.split()
                assert left_ == "*Vertices"
                vertex_count_ = int(right_)
                state_ = "vertex-ids"
            elif state_ == "vertex-ids":
                left_, *right_ = line_.split()
                vertex_name_ = " ".join(right_).replace("\"", "")
                vertex_id_: int = int(left_)
                vertex_names_[vertex_id_] = vertex_name_
                network_[vertex_name_] = set()
                if vertex_id_ == vertex_count_:
                    state_ = "arcs-title"
            elif state_ == "arcs-title":
                assert line_ == "*Arcslist"
                state_ = "arcs-list"
            elif state_ == "arcs-list":
                # noinspection PyTypeChecker
                nodes_ = map(vertex_names_.get, map(int, line_.split()))
                for left_node_, right_node_ in itertools.permutations(nodes_, 2):
                    if left_node_ != right_node_:
                        network_[left_node_].add(right_node_)
    return network_


roget_network: Network = load_roget("./topic3networks/Roget.txt")

In [8]:
def load_ccsb_y2h(filename_) -> Network:
    network_: Network = {}
    with open(filename_, "rt") as file_:
        state_: Literal["first-line", "edges"] = "first-line"
        for line_ in file_:
            line_ = line_.rstrip()
            if state_ == "first-line":
                assert line_.startswith("from")
                state_ = "edges"
            elif state_ == "edges":
                left_node_, right_node_, *_ = line_.split("\t")
                if left_node_ not in network_:
                    network_[left_node_] = set()
                if right_node_ not in network_:
                    network_[right_node_] = set()
                if left_node_ != right_node_:
                    network_[left_node_].add(right_node_)
                    network_[right_node_].add(left_node_)
    return network_


ccsb_y2h_network: Network = load_ccsb_y2h("./topic3networks/CCSB-Y2H.txt")

Now that the networks are loaded, compute the largest connected component for each one.

In [9]:
lcc_ltn = compute_largest_connected_component(london_transport_network)
print(f"London Transport Network largest connected component size: "
      f"{len(lcc_ltn)}/{len(london_transport_network)}")

100%|██████████| 369/369 [00:00<00:00, 368850.85it/s]

London Transport Network largest connected component size: 369/369


Next, remove the nodes from each network not in its largest connected component.

In [10]:
lcn_ltn = keep_nodes(london_transport_network, lcc_ltn)

100%|██████████| 369/369 [00:00<?, ?it/s]


Compute the centralities for each network.

In [11]:
centralities_ltn = compute_centralities(lcn_ltn)

100%|██████████| 369/369 [00:00<00:00, 1892.32it/s]


In [12]:
print(f"Top 20 Roget nodes by centrality measure:\n"
      f"{format_centralities(*centralities_ltn)}")

Top 20 Roget nodes by centrality measure:
Closeness Centrality:
1         greenpark       - 0.000307
2        westminster      - 0.000299
3         bondstreet      - 0.000299
4    kingscrossstpancras  - 0.000299
5        oxfordcircus     - 0.000298
6            bank         - 0.000295
7          waterloo       - 0.000295
8        bakerstreet      - 0.000294
9           euston        - 0.000291
10         victoria       - 0.000290
11        farringdon      - 0.000290
12          angel         - 0.000290
13      hydeparkcorner    - 0.000288
14         moorgate       - 0.000286
15         barbican       - 0.000285
16        oldstreet       - 0.000285
17       warrenstreet     - 0.000284
18     liverpoolstreet    - 0.000284
19    highbury&islington  - 0.000284
20     piccadillycircus   - 0.000284
21       eustonsquare     - 0.000284
22         holborn        - 0.000282
23        embankment      - 0.000282

Nearness Centrality:
1         greenpark       - 59.520149
2            bank        

Do the same for the Roget network...

In [13]:
lcc_roget = compute_largest_connected_component(roget_network)
print(f"Roget Network largest connected component size: "
      f"{len(lcc_roget)}/{len(roget_network)}")

100%|██████████| 1022/1022 [00:00<00:00, 340826.80it/s]

Roget Network largest connected component size: 994/1022


In [14]:
lcn_roget = keep_nodes(roget_network, lcc_roget)

100%|██████████| 1022/1022 [00:00<00:00, 255472.83it/s]


In [15]:
centralities_roget = compute_centralities(lcn_roget)

100%|██████████| 994/994 [00:03<00:00, 321.47it/s]


In [16]:
print(f"Top 20 Roget nodes by centrality measure:\n"
      f"{format_centralities(*centralities_roget)}")

Top 20 Roget nodes by centrality measure:
Closeness Centrality:
1         inutility       - 0.000496
2           store         - 0.000491
3          neglect        - 0.000490
4       deterioration     - 0.000490
5           truth         - 0.000489
6        unimportance     - 0.000489
7        unconformity     - 0.000488
8          support        - 0.000487
9         indication      - 0.000484
10        inactivity      - 0.000484
11        deception       - 0.000482
12         activity       - 0.000481
13           aid          - 0.000481
14        restraint       - 0.000481
15           care         - 0.000480
16          skill         - 0.000478
17       information      - 0.000477
18       preparation      - 0.000477
19           plan         - 0.000475
20        hindrance       - 0.000475
21     pleasurableness    - 0.000475
22         violence       - 0.000475

Nearness Centrality:
1         inutility       - 540.250000
2          neglect        - 533.500000
3       deterioration 

Finally, do the same for the CCSB-Y2H network...

In [17]:
lcc_ccsb = compute_largest_connected_component(ccsb_y2h_network)
print(f"CCSB-Y2H Network largest connected component size: "
      f"{len(lcc_ccsb)}/{len(ccsb_y2h_network)}")

100%|██████████| 1278/1278 [00:00<00:00, 425929.32it/s]

CCSB-Y2H Network largest connected component size: 964/1278


In [18]:
lcn_ccsb = keep_nodes(ccsb_y2h_network, lcc_ccsb)

100%|██████████| 1278/1278 [00:00<00:00, 425963.17it/s]


In [19]:
centralities_ccsb = compute_centralities(lcn_ccsb)

100%|██████████| 964/964 [00:01<00:00, 729.20it/s]


In [20]:
print(f"Top 20 CCSB-Y2H nodes by centrality measure:\n"
      f"{format_centralities(*centralities_ccsb)}")

Top 20 CCSB-Y2H nodes by centrality measure:
Closeness Centrality:
1          YLR291C        - 0.000336
2          YBR261C        - 0.000297
3          YPL070W        - 0.000295
4          YCL028W        - 0.000292
5          YPL049C        - 0.000289
6          YLR423C        - 0.000289
7          YNL044W        - 0.000283
8          YHR113W        - 0.000282
9          YOR284W        - 0.000282
10         YLR245C        - 0.000280
11         YBR080C        - 0.000279
12         YPL088W        - 0.000278
13         YGR267C        - 0.000277
14         YHL018W        - 0.000277
15         YBR233W        - 0.000276
16         YMR095C        - 0.000276
17         YDR256C        - 0.000276
18         YOR095C        - 0.000275
19         YHR112C        - 0.000275
20         YKR034W        - 0.000275
21         YGL153W        - 0.000275
22         YDL239C        - 0.000274

Nearness Centrality:
1          YLR291C        - 384.588095
2          YLR423C        - 327.497619
3          YBR261C 

## Comments on findings

### Closeness Centrality vs Nearness Centrality

### Degree Centrality Adjacency Centrality